In [4]:
import numpy as np
import pandas as pd
import datetime as dt

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from flask import Flask, jsonify

In [12]:
# Database Setup
database_path = "./Resources/hawaii.sqlite"
engine = create_engine(f"sqlite:///{database_path}", echo=False)

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save reference to the table
Measurements = Base.classes.measurement
Stations = Base.classes.station

# Flask Setup
app = Flask(__name__)
session = Session(engine)


In [3]:
# Flask Routes
-----------------------------------------------------
@app.route("/")
def welcome():
    """List all available api routes."""
    session = Session(engine)

    latestDate = session.query(Measurements.date).order_by(Measurements.date.desc()).first()
    earliestDate = session.query(Measurements.date).order_by(Measurements.date).first()
   
    session.close()
    
    return (
        f"Available Routes:<br/>"
        f"&emsp;&emsp;/api/v1.0/precipitation<br/>"
        f"&emsp;&emsp;/api/v1.0/stations</br>"
        f"&emsp;&emsp;/api/v1.0/tobs<br/>"
        f"<br/>"
        f"Available Date Search Routes:<br/>"
        f"&emsp;Data Available between:&emsp;{earliestDate[0]}&emsp; to&emsp; {latestDate[0]}<br/>"
        f"&emsp;*format dates as 'yyyy-mm-dd'*<br/>"
        f"&emsp;&emsp;/api/v1.0/startdate<br/>"
        f"&emsp;&emsp;/api/v1.0/startdate/enddate"
    )
-----------------------------------------------------
@app.route("/api/v1.0/precipitation")
def precipitation():
    # Create our session (link) from Python to the DB
    session = Session(engine)

    # Query precipitation data
    
    query_date = dt.date(2017, 8, 23) - dt.timedelta(days=365)
    precipScores = session.query(Measurements.station, Measurements.date, Measurements.prcp).filter(Measurements.date >= query_date).\
    order_by(Measurements.date).all()
    session.close()

    
    prcpScores_df = pd.DataFrame(precipScores, columns=['Stations','Date','Precipitation'])
    prcpSums_df = prcpScores_df.groupby("Date").sum()
    


    # Convert list of tuples into normal list
    prcpData = [prcpSums_df.to_dict()]
    
    return jsonify(prcpData)
-----------------------------------------------------
@app.route("/api/v1.0/stations")
def stations():
    # Create our session (link) from Python to the DB
    session = Session(engine)

    # Query station data 
    activeStations = session.query(Stations.station,Stations.name,func.count(Measurements.station)).\
        filter(Stations.station == Measurements.station).\
        group_by(Measurements.station).\
        order_by(func.count(Measurements.station).desc()).all()    
    
    session.close()
    
    stationList =[]

    for station, name, observations in activeStations:
        station_dict = {}
        station_dict['StationID'] = station
        station_dict['Name'] = name
        station_dict['Observations'] = observations
        stationList.append(station_dict)    
    
    return jsonify(stationList)
-----------------------------------------------------
@app.route("/api/v1.0/tobs")
def temperature():
    # Create our session (link) from Python to the DB
    session = Session(engine)
    
    # Query date
    query_date = dt.date(2017, 8, 23) - dt.timedelta(days=365)

    # Query to find most active station ID
    activeStations =session.query(Stations.station, Stations.name,func.count(Measurements.station)).\
        filter(Stations.station == Measurements.station).\
        group_by(Measurements.station).\
        order_by(func.count(Measurements.station).desc()).all()
    
    topStation =  [result[0] for result in activeStations][0]
    
    # Query most active station data 
    topActive = session.query(Measurements.station, Stations.name, Measurements.date, Measurements.tobs).\
        filter(Stations.station == Measurements.station).\
        filter(Measurements.station == topStation).\
        filter(Measurements.date >= query_date).\
        order_by(Measurements.date).all()
    
    session.close()

    # Create dictionary for temp data of most active station
    topActive_df = pd.DataFrame(topActive, columns=["StationID","Name","Date","Temperature"])
    stationTemp_dict = topActive_df[["Date","Temperature"]].set_index("Date").to_dict()
    stationTemps= stationTemp_dict["Temperature"]
    
    # Strip most active station name and ID
    stationID = [result[0] for result in topActive][1]
    stationName = [result[1] for result in topActive][1]
    
    # Create dictionary for most active station
    activeStation = [{"Name":stationName,
                     "Station ID":stationID,
                     "Temperatures":stationTemps
                    }]
    
    return jsonify(activeStation)
-----------------------------------------------------
@app.route("/api/v1.0/<start>")
def stats_by_start_date(start):
    # Create our session (link) from Python to the DB
    session = Session(engine)
    
    # Calculate query date
    search = dt.date.fromisoformat(start)

    # Query for temp data
    stationData = session.query(Measurements.station, Stations.name, Measurements.date, Measurements.tobs).\
        filter(Stations.station == Measurements.station).\
        filter(Measurements.date >= search).\
        order_by(Measurements.date).all()
    
    session.close()

    # Calculate statistics from search
    stationData_df = pd.DataFrame(stationData, columns=["StationID","Name","Date","Temperature"])
    tests = ['min','max','mean']
    stationStats_df = stationData_df.groupby(["StationID","Name"]).agg({"Temperature":tests}).reset_index() 
    
    # Create dictionary data from search
    tempResults = [] 
    
    for x in range(len(stationStats_df)):
        stationStats = {}     
        stationStats["Name"] = stationStats_df.iloc[x,1]
        stationStats["Station ID"] =stationStats_df.iloc[x,0]
        stationStats["Temperatures"] = stationStats_df["Temperature"].iloc[x,:].to_dict()
        tempResults.append(stationStats)
    
    return jsonify(tempResults)

-----------------------------------------------------

@app.route("/api/v1.0/<start>/<end>")
def stats_between_dates(start,end):


In [ ]:
@app.route("/api/v1.0/<start>/<end>")
def stats_between_dates(start,end):
    # Create our session (link) from Python to the DB
    session = Session(engine)
    
    # Calculate query date
    searchStart = dt.date.fromisoformat(start)
    searchEnd = dt.date.fromisoformat(end)
    dateTest = startDate <= endDate
    
    if dateTest == False:
        return jsonify({"ERROR": "End date should be greater than start date."}), 404
        
    # Query for temp data
    stationData = session.query(Measurements.station, Stations.name, Measurements.date, Measurements.tobs).\
        filter(Stations.station == Measurements.station).\
        filter(Measurements.date >= startDate).\
        filter(Measurements.date <= endDate).\
        order_by(Measurements.date).all()
    
    session.close()

    # Calculate statistics from search
    stationData_df = pd.DataFrame(stationData, columns=["StationID","Name","Date","Temperature"])
    tests = ['min','max','mean']
    stationStats_df = stationData_df.groupby(["StationID","Name"]).agg({"Temperature":tests}).reset_index() 
    
    # Create dictionary data from search
    tempResults = [] 
    
    for x in range(len(stationStats_df)-1):
        stationStats = {}     
        stationStats["Name"] = stationStats_df.iloc[x,1]
        stationStats["Station ID"] =stationStats_df.iloc[x,0]
        stationStats["Temperatures"] = stationStats_df["Temperature"].iloc[x,:].to_dict()
        tempResults.append(stationStats)
    
    return jsonify(tempResults)

In [64]:
start = f"2016-08-23"
end =f"2017-08-23"

In [84]:
startDate = dt.date.fromisoformat(start)
endDate = dt.date.fromisoformat(end)

In [86]:
dateTest = startDate <= endDate

if dateTest == True:
    print(dateTest)

True


In [95]:
stationData = session.query(Measurements.station, Stations.name, Measurements.date, Measurements.tobs).\
        filter(Stations.station == Measurements.station).\
        filter(Measurements.date >= startDate).\
        filter(Measurements.date <= endDate).\
        order_by(Measurements.date).limit(10).all()
stationData

stationData_df = pd.DataFrame(stationData, columns=["StationID","Name","Date","Temperature"])

tests = ['min','max','mean']
stationStats_df = stationData_df.groupby(["StationID","Name"]).agg({"Temperature":tests}).reset_index()
stationStats_df

StationID                                    Name Temperature            
                                                               min   max  mean
0  USC00513117                    KANEOHE 838.1, HI US        76.0  76.0  76.0
1  USC00514830  KUALOA RANCH HEADQUARTERS 886.9, HI US        80.0  80.0  80.0
2  USC00516128            MANOA LYON ARBO 785.2, HI US        74.0  74.0  74.0
3  USC00517948                       PEARL CITY, HI US        80.0  80.0  80.0
4  USC00519281                     WAIHEE 837.5, HI US        77.0  77.0  77.0
5  USC00519397                    WAIKIKI 717.2, HI US        79.0  81.0  80.0
6  USC00519523      WAIMANALO EXPERIMENTAL FARM, HI US        80.0  80.0  80.0

In [63]:
tempResults = [] 
    
for x in range(len(stationStats_df)):
    stationStats = {}     
    stationStats["Name"] = stationStats_df.iloc[x,1]
    stationStats["Station ID"] =stationStats_df.iloc[x,0]
    stationStats["Temperatures"] = stationStats_df["Temperature"].iloc[x,:].to_dict()
    tempResults.append(stationStats)
tempResults

[{'Name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US',
  'Station ID': 'USC00514830',
  'Temperatures': {'min': 82.0, 'max': 82.0, 'mean': 82.0}},
 {'Name': 'MANOA LYON ARBO 785.2, HI US',
  'Station ID': 'USC00516128',
  'Temperatures': {'min': 76.0, 'max': 76.0, 'mean': 76.0}},
 {'Name': 'WAIKIKI 717.2, HI US',
  'Station ID': 'USC00519397',
  'Temperatures': {'min': 81.0, 'max': 81.0, 'mean': 81.0}},
 {'Name': 'WAIMANALO EXPERIMENTAL FARM, HI US',
  'Station ID': 'USC00519523',
  'Temperatures': {'min': 82.0, 'max': 82.0, 'mean': 82.0}}]

In [52]:
test = stationStats_df[["StationID","Temperature"]]
stationStats_df.iloc[2,0]

'USC00516128'

In [57]:
stationStats_df["Temperature"].iloc[0,:].to_dict()

{'min': 61.0, 'max': 82.0, 'mean': 73.27696793002916}

In [87]:
    latestDate = session.query(Measurements.date).order_by(Measurements.date.desc()).first()
    earliestDate = session.query(Measurements.date).order_by(Measurements.date).first()

In [93]:
test = list(latestDate)
test[0]

'2017-08-23'

In [94]:
earliestDate[0]

'2010-01-01'